### Jesus Olivera
#### Independent Study: YU Data Labeling Tool
<br>


# Tweets Sentiment Analysis 

## Script Logic

1. Import required libraries
2. Import tweets data from S3 
3. Instantiate the sentiment intensity analyzer, parse tweets and score sentiment
4. Create dataframe with negative tweets
5. Upload negative tweets dataframe to S3

In [32]:
# Import libraries
import nltk
nltk.download('vader_lexicon')
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# NLTK VADER for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import sys
import numpy as np
from timeit import timeit
import boto3
import s3fs
import datetime as dt

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [36]:
# Retrieving dataset from S3

tweets_df = pd.read_csv('s3://tweetsdatatest/Tweets train.csv')
tweets_df.head(5)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [37]:
# Instantiate the sentiment intensity analyzer
vader = SentimentIntensityAnalyzer()

# Set column names
columns = ['id', 'label', 'tweet']

# Convert the parsed_news list into a DataFrame called 'parsed_and_scored_news'
parsed_and_scored_tweets = pd.DataFrame(tweets_df, columns=columns)

# Iterate through the headlines and get the polarity scores using vader
scores = parsed_and_scored_tweets['tweet'].apply(vader.polarity_scores).tolist()

# Convert the 'scores' list of dicts into a DataFrame
scores_df = pd.DataFrame(scores)

# Join the DataFrames of the news and the list of dicts
parsed_and_scored_tweets = parsed_and_scored_tweets.join(scores_df, rsuffix='_right')


parsed_and_scored_tweets.head()

,id,label,tweet,neg,neu,pos,compound
0,1,0,@user when a father is dysfunctional and is s...,0.385,0.615,0.000,-0.8296
1,2,0,@user @user thanks for #lyft credit i can't us...,0.000,0.744,0.256,0.6705
2,3,0,bihday your majesty,0.000,1.000,0.000,0.0000
3,4,0,#model i love u take with u all the time in ...,0.000,0.663,0.337,0.7249
4,5,0,factsguide: society now #motivation,0.000,1.000,0.000,0.0000


In [34]:
# Creating dataframe with negative tweets
neg_tweets = parsed_and_scored_tweets[(parsed_and_scored_tweets.neg > 0 )].reset_index(drop=True)

neg_tweets.head(5)

,id,label,tweet,neg,neu,pos,compound
0,1,0,@user when a father is dysfunctional and is s...,0.385,0.615,0.000,-0.8296
1,6,0,[2/2] huge fan fare and big talking before the...,0.323,0.500,0.177,-0.4215
2,12,0,we are so selfish. #orlando #standwithorlando ...,0.250,0.750,0.000,-0.6113
3,15,1,no comment! in #australia #opkillingbay #se...,0.238,0.762,0.000,-0.3595
4,21,0,#euro2016 people blaming ha for conceded goal ...,0.111,0.692,0.197,0.3612


In [35]:
# creating a variable that stores date to be append on the file to be archive
today = dt.datetime.today()
date = today.strftime("%Y%m%d")

#connect and load the historical data file to s3

s3pathName = 's3://neg-tweets/'
FileNameFullPath = 'neg_tweets' 
filenames3 = "%s%s%s.csv"%(s3pathName,FileNameFullPath,date) #name of the filepath and csv file

neg_tweets.to_csv(filenames3, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3://neg-tweets/neg_tweets20220406.csv
